Zomato Restaurant

In [ ]:
# importing the required libraries 
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectPercentile
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# df is the zomato csv and where as df1 represents the country. 
df = pd.read_csv('https://raw.githubusercontent.com/dsrscientist/dataset4/main/zomato.csv', encoding = "ISO-8859-1")
df1 = pd.read_csv(r'C:/\Users\Country Head\anaconda3\Zomato.csv')

In [ ]:
# first 5 rows and all columns
df.head()

In [ ]:
# Checking the shape of the first dataframe
df.shape

In [ ]:
# analysing the second dataframe
df1.head(14)

In [ ]:
# the shape of 2nd dataframe.
df1.shape

In [ ]:
# merging both the datset based on the country code - the common parametre
df = pd.merge(df, df1, on='Country Code')

In [ ]:
# analyzing the final dataframe.
df

In [ ]:
# Select features
X = df.drop(['Average Cost for two'], axis=1)
y = df['Average Cost for two']

In [ ]:
df.isnull().sum().to_frame("No. of null values")

In [ ]:
sns.heatmap(df.isnull())

In [ ]:

# Count unique occurrences of each location
Country_counts = df['Country'].value_counts()



# Plotting the bar chart representing the locations
plt.figure(figsize=(15,10))
Country_counts.plot(kind='bar')
plt.title('Bar chart of Locations')
plt.xlabel('Country')
plt.ylabel('Count')
plt.show()



 - representing the no null values in the dataset.

In [ ]:
# checking the columns of the df
df.columns

In [ ]:
# understanding the dataframe.
df.info()

There are two types of data in the available in the datasete 
- object 
- int 64

In [ ]:
# Check for the unique values and the count. 
for i in df.columns: 
    print(f"columns:{i}")
    print(df[i].value_counts())
    print("n")

In [ ]:
# We shall drop the Longitude and latitude since it is not adding any value to the output. 
df = df.drop(['Longitude', 'Latitude'], axis=1)

In [ ]:
# Categorizing columns as categorical or numerical
categorical_col = []
numerical_col = []


for i in df.columns:
    if df.dtypes[i] == "object":
        categorical_col.append(i)
    else:
        numerical_col.append(i)

print("Categorical Columns:", categorical_col)
print("\n")
print("Numerical Columns:", numerical_col)

In [ ]:
# Encoding the values of the categorical column
le = LabelEncoder()
for col in categorical_col:
    df[col] = le.fit_transform(df[col])

In [ ]:
# Checking for the outliers in the dataframe.
plt.figure(figsize =(10,6))
plotnumber = 1

for col in numerical_col:
    if plotnumber<=9:
        ax = plt.subplot(4,5,plotnumber)
        sns.boxplot(df[col])
        plt.xlabel(col,fontsize=15)
        plt.xticks(rotation=0,fontsize=10)
    plotnumber+=1
plt.tight_layout()

In [ ]:
# Checking the skewness of the categorical column. 
df[numerical_col].skew()

In [ ]:
### import pandas as pd
from scipy.stats import yeojohnson
# creating the a dataframe for the columns with skewness. 
skew_cols = ['Country Code', 'Aggregate rating', 'Votes','Country','Price range','Is delivering now',]

# Applying to apply the Yeo-Johnson transformation method to the above df. 
for cols in skew_cols:
    df[cols], _ = yeojohnson(df[cols])

# Check the skewness after transformation
print(df[skew_cols].skew())

Working on the outliers using the Z score method and the IQR Method. 

In [ ]:
from scipy.stats import zscore 
feature_outliers = df[['Votes','Country Code','Aggregate rating','Average Cost for two','Price range']]
z=np.abs(zscore(feature_outliers))
z

In [ ]:
#keeping the threshold=3
np.where(z>3)

In [ ]:
z.iloc[9397,1]

In [ ]:
# assigning a new df for the values. 
df_new = df[(z < 3).all(axis=1)]
print(df_new.shape)

In [ ]:
#splitting the Feature and the target. 
X = df.drop(columns='Average Cost for two',axis=1)
Y = df['Average Cost for two']

#Calculating z-scores
z = np.abs(zscore(X))

#Filtering features and target for z-scores less than 3
X1 = X[(z < 3).all(axis=1)]
Y1 = Y[(z < 3).all(axis=1)]

In [ ]:
# checking the Shape of old and new dataframe
print("old Data Frame : ",df.shape[0])
print("New Data Frame : ",df_new.shape[0])

In [ ]:
print("Data Loss Percentage = ",((df.shape[0]-df_new.shape[0])/df.shape[0])*100)

In [ ]:
le = LabelEncoder()
for col in categorical_col:
    df[col] = le.fit_transform(df[col])

In [ ]:
# We can see that the data loss percentange is high therefore we shall proceed without lossing the data.

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(100, 80))
sns.pairplot(df,hue="Average Cost for two",palette="Set1")

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df.corr(), annot=True, fmt ="0.2", linecolor = "black",)
plt.title("Correlation graph")
plt.xlabel('figure',fontsize=14)
plt.ylabel('feature_label',fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(22, 7))
df.corr()['Average Cost for two'].sort_values(ascending=False).drop('Average Cost for two').plot(kind='bar', color="m")
plt.xlabel('Feature', fontsize=15)
plt.ylabel('Correlation with income', fontsize=15)
plt.title('Correlation between Features and Income', fontsize=20)
plt.show()

In [ ]:
from sklearn.feature_selection import SelectPercentile, chi2
SPercentile = SelectPercentile(score_func=chi2, percentile=80)
SPercentile = SPercentile.fit(X,y)

In [ ]:
# seperate the function to check the p value 
cols = SPercentile.get_support(indices=True)
print ('Feature Index = ',cols)

features = X.columns[cols]
print('Features = ',list(features))

In [ ]:
df_scores = pd.DataFrame({'features': X.columns, 'Chi2Score': SPercentile.scores_, 'pValue': SPercentile.pvalues_})
sorted_df_scores = df_scores.sort_values(by='Chi2Score', ascending=False)
print(sorted_df_scores)

In [ ]:
df.head()

In [ ]:
X = df[features] # Remove the square brackets around 'W' and fix the axis argument
y = df['Average Cost for two']

In [ ]:
# Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Perform feature scaling using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Instantiate and fitting the model

lr = LinearRegression()
lr.fit(X_train, y_train)

rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

gr = GradientBoostingRegressor()
gr.fit(X_train, y_train)

dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

In [ ]:
# Prection on the test data. 
y_pred1 = lr.predict(X_test)
y_pred2 = rfr.predict(X_test)
y_pred3 = gr.predict(X_test)
y_pred4 = dt.predict(X_test)
df2 = pd.DataFrame({'Actual': y_test, 'Lr': y_pred1, 'rfr': y_pred2, 'gr': y_pred3,'dt':y_pred4,})

In [ ]:
# Evaluating the models
lr_mse = mean_squared_error(y_test, lr_predictions)
rfr_mse = mean_squared_error(y_test, rfr_predictions)
gr_mse = mean_squared_error(y_test, gr_predictions)
dt_mse = mean_squared_error(y_test, dt_predictions)

lr_mae = mean_absolute_error(y_test, lr_predictions)
rfr_mae = mean_absolute_error(y_test, rfr_predictions)
gr_mae = mean_absolute_error(y_test, gr_predictions)
dt_mae = mean_absolute_error(y_test, dt_predictions)

lr_r2 = r2_score(y_test, lr_predictions)
rfr_r2 = r2_score(y_test, rfr_predictions)
gr_r2= r2_score(y_test, gr_predictions)
dt_r2 = r2_score(y_test, dt_predictions)

In [ ]:
df2

In [ ]:
plt.subplot(221)
plt.plot(df2['Actual'].iloc[0:10], label='Actual')
plt.plot(df2['rfr'].iloc[0:10], label="rfr")
plt.xlabel('Data Point')
plt.ylabel('Value')
plt.title('Comparison - Actual vs rfr')
plt.legend()

# Repeat the above code for other subplots (222, 223, 224)
plt.subplot(222)
plt.plot(df2['Actual'].iloc[0:10], label='Actual')
plt.plot(df2['Lr'].iloc[0:10], label="Lr")
plt.xlabel('Data Point')
plt.ylabel('Value')
plt.title('Comparison - Actual vs Lr')
plt.legend()

plt.subplot(223)
plt.plot(df2['Actual'].iloc[0:10], label='Actual')
plt.plot(df2['gr'].iloc[0:10], label="gr")
plt.xlabel('Data Point')
plt.ylabel('Value')
plt.title('Comparison - Actual vs gr')
plt.legend()

plt.subplot(224)
plt.plot(df2['Actual'].iloc[0:10], label='Actual')
plt.plot(df2['dt'].iloc[0:10], label="dt")
plt.xlabel('Data Point')
plt.ylabel('Value')
plt.title('Comparison - Actual vs dt')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Calculating mean squared error (MSE) for each model
mse_lr = mean_squared_error(df2['Actual'], df2['Lr'])
mse_rfr = mean_squared_error(df2['Actual'], df2['rfr'])
mse_gr = mean_squared_error(df2['Actual'], df2['gr'])
mse_dt = mean_squared_error(df2['Actual'], df2['dt'])

# dictionary that stores the MSE values
mse_scores = {
    'Linear Regression': mse_lr,
    'Random Forest': mse_rfr,
    'Gradient Boosting': mse_gr,
    'Decision Tree': mse_dt}

In [ ]:
from sklearn.model_selection import cross_val_score

# Assuming 'lr', 'rfr', 'gr', 'dt' are your trained models
models = {'Linear Regression': lr, 'Random Forest Regressor': rfr, 'Gradient Boosting Regressor': gr, 'Decision Tree Regressor': dt}

for model_name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f"Cross-Validation Scores for {model_name}: ", scores)
    print(f"Average Cross-Validation Score for {model_name}: ", scores.mean())


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

rfr = RandomForestClassifier(random_state=0)

# Define a grid of hyperparameter 'params_
param_grid = {
    'n_estimators': [50, 100, 200], 
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_rf = GridSearchCV(estimator=rfr, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

grid_rf.fit(X_train, y_train)
print("Best parameters found: ", grid_rf.best_params_)
print("Best score found: ", grid_rf.best_score_)

In [ ]:
# Get the best parameters
best_params = grid_rf.best_params_
print("Best parameters found: ", best_params)

# Get the best score
best_score = grid_rf.best_score_
print("Best score found: ", best_score)

In [ ]:
# To find the best model 
best_rf = min(mse_scores, key=mse_scores.get)
best_rf.fit(X_train, y_train)
print("The best model is:", best_model)

In [ ]:
data_Predict= {'Restaurant ID':[6304287],
               'Restaurant Name':[5523],
               'Country Code': [162],
               'City' :[75],
               'Address':[4684],
               'Locality':[398],
               'Locality Verbose': [314],
               'Cuisines': [1111],
               'Average Cost for two': [1200],
               'Currency': [0],
               'Has Table booking': [1],
               'Has Online delivery':[0],
               'Is delivering now':[1],
               'Switch to order menu':[1],
               'Price range' :[4],
               'Aggregate rating':[4.9],
               'Rating color':[1],
               'Rating text':[1],
               'Votes': [365],
               'Country': [6]}

df_pred = pd.DataFrame(data_Predict,index=[0])
df_pred

In [ ]:
# Predicting the result using the best model. 
new_pred = best_rfr.predict(df_pred)
print(new_pred)

In [ ]:
from joblib import dump, load

# saving the model. 
dump(best_rf, 'best_rf.joblib') 

# Loading the model
best_rf_from_joblib = load('best_rf.joblib') 

# using the loaded model to make predictions
best_rf_from_joblib.predict(X1_test)

- Price range ( 2nd Prediction)

In [ ]:
# for the Next Tmin value 
X2 = df.drop(['Price range'],axis=1)
y2 = df['Price range']

In [ ]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X2_train_scaled = scaler.fit_transform(X2_train)
X2_test_scaled = scaler.transform(X2_test)

In [ ]:
# Splitting the data
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [ ]:
# defining and fitting the model 
lr = LinearRegression()
lr.fit(X2_train_scaled, Y2_train)

rfr = RandomForestRegressor()
rfr.fit(X2_train_scaled, Y2_train)

gr = GradientBoostingRegressor()
gr.fit(X2_train_scaled, Y2_train)

dt = DecisionTreeRegressor()
dt.fit(X2_train_scaled, Y2_train)

In [ ]:
# Making predictions on the test data
lr_predictions = lr.predict(X2_test)
rfr_predictions = rfr.predict(X2_test)
gr_predictions = gr.predict(X2_test)
dt_predictions = dt.predict(X2_test)

# Evaluating the models
lr_mse2 = mean_squared_error(Y2_test, lr_predictions)
rfr_mse2 = mean_squared_error(Y2_test, rfr_predictions)
gr_mse2 = mean_squared_error(Y2_test, gr_predictions)
dt_mse2 = mean_squared_error(Y2_test, dt_predictions)

lr_mae2 = mean_absolute_error(Y2_test, lr_predictions)
rfr_mae2 = mean_absolute_error(Y2_test, rfr_predictions)
gr_mae2 = mean_absolute_error(Y2_test, gr_predictions)
dt_mae2 = mean_absolute_error(Y2_test, dt_predictions)

lr_r2_2 = r2_score(Y2_test, lr_predictions)
rfr_r2_2 = r2_score(Y2_test, rfr_predictions)
gr_r2_2= r2_score(Y2_test, gr_predictions)
dt_r2_2 = r2_score(Y2_test, dt_predictions)

In [ ]:
# Print Mean Squared Error for each model
print("Linear Regression MSE: ", lr_mse2)
print("Random Forest Regressor MSE: ", rfr_mse2)
print("Gradient Boosting Regressor MSE: ", gr_mse2)
print("Decision Tree Regressor MSE: ", dt_mse2)

# Print Mean Absolute Error for each model
print("\nLinear Regression MAE: ", lr_mae2)
print("Random Forest Regressor MAE: ", rfr_mae2)
print("Gradient Boosting Regressor MAE: ", gr_mae2)
print("Decision Tree Regressor MAE: ", dt_mae2)

# Print R2 Score for each model
print("\nLinear Regression R2 Score: ", lr_r2_2)
print("Random Forest Regressor R2 Score: ", rfr_r2_2)
print("Gradient Boosting Regressor R2 Score: ", gr_r2_2)
print("Decision Tree Regressor R2 Score: ", dt_r2_2)

In [ ]:
# Create a table to compare the scores
data = {
    'Model': ['Linear Regression', 'Random Forest', 'Gradient Boosting', 'Decision Tree'],
    'Mean Squared Error (MSE)': [lr_mse2, rfr_mse2, gr_mse2, dt_mse2],
    'Mean Absolute Error (MAE)': [lr_mae2, rfr_mae2, gr_mae2, dt_mae2],
    'R^2 Score': [lr_r2_2, rfr_r2_2, gr_r2_2, dt_r2_2]}

df = pd.DataFrame(data)

# Print the DataFrame
print(df)

In [ ]:
# Prection on the test data. 
y_pred1 = lr.predict(X2_test)
y_pred2 = rfr.predict(X2_test)
y_pred3 = gr.predict(X2_test)
y_pred4 = dt.predict(X2_test)
df3 = pd.DataFrame({'Actual': Y2_test, 'Lr': y_pred1, 'rfr': y_pred2, 'gr': y_pred3,'dt':y_pred4,})

In [ ]:
df3

In [ ]:
# For rfr2 comparison
plt.subplot()
plt.plot(df3['Actual'].iloc[0:10], label='Actual')
plt.plot(df3['rfr'].iloc[0:10], label="rfr")
plt.xlabel('Data Point')
plt.ylabel('Value')
plt.title('Comparison - Actual vs rfr')
plt.legend()

# For Lr2 comparison
plt.subplot()
plt.plot(df3['Actual'].iloc[0:10], label='Actual')
plt.plot(df3['Lr'].iloc[0:10], label="Lr")
plt.xlabel('Data Point')
plt.ylabel('Value')
plt.title('Comparison - Actual vs Lr')
plt.legend()

# For gr2 comparison
plt.subplot()
plt.plot(df3['Actual'].iloc[0:10], label='Actual')
plt.plot(df3['gr'].iloc[0:10], label="gr")
plt.xlabel('Data Point')
plt.ylabel('Value')
plt.title('Comparison - Actual vs gr')
plt.legend()

# For dt2 comparison
plt.subplot()
plt.plot(df3['Actual'].iloc[0:10], label='Actual')
plt.plot(df3['dt'].iloc[0:10], label="dt")
plt.xlabel('Data Point')
plt.ylabel('Value')
plt.title('Comparison - Actual vs dt')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Calculating mean squared error (MSE) for each model
mse_lr = mean_squared_error(df3['Actual'], df3['Lr'])
mse_rfr = mean_squared_error(df3['Actual'], df3['rfr'])
mse_gr = mean_squared_error(df3['Actual'], df3['gr'])
mse_dt = mean_squared_error(df3['Actual'], df3['dt'])

# dictionary that stores the MSE values
mse_scores = {'Linear Regression': mse_lr,
    'Random Forest': mse_rfr,
    'Gradient Boosting': mse_gr,
    'Decision Tree': mse_dt}

# To find the best model 
best_model = min(mse_scores, key=mse_scores.get)

print("The best model is:", best_model)

In [ ]:
#Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4, 6]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X2_train, Y2_train)

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)
print()

# Train the best model on the entire training set
best_model.fit(X2_train, Y2_train)

# Evaluate the best model on the testing set
best_predictions = best_model.predict(X2_test)
best_mse = mean_squared_error(Y2_test, best_predictions)
best_mae = mean_absolute_error(Y2_test, best_predictions)
best_r2 = r2_score(Y2_test, best_predictions)

print("Best Model Evaluation:")
print("MSE:", best_mse)
print("MAE:", best_mae)
print("R-squared:", best_r2)

In [ ]:
# Create a table to compare the scores
data = {
    'Model': ['Linear Regression', 'Random Forest', 'Gradient Boosting', 'Decision Tree'],
    'Mean Squared Error (MSE)': [lr_mse2, rfr_mse2, gr_mse2, dt_mse2],
    'Mean Absolute Error (MAE)': [lr_mae2, rfr_mae2, gr_mae2, dt_mae2],
    'R^2 Score': [lr_r2_2, rfr_r2_2, gr_r2_2, dt_r2_2]}

df = pd.DataFrame(data)



In [ ]:
# Print the DataFrame
print(df)

In [ ]:
# Get the best parameters
best_params = grid_rf.best_params_
print("Best parameters found: ", best_params)

# Get the best score
best_score = grid_rf.best_score_
print("Best score found: ", best_score)

In [ ]:
df.head()

In [ ]:
data_Predict= {'Restaurant ID':[6304287],
               'Restaurant Name':[5523],
               'Country Code': [162],
               'City' :[75],
               'Address':[4684],
               'Locality':[398],
               'Locality Verbose': [314],
               'Cuisines': [1111],
               'Average Cost for two': [1200],
               'Currency': [0],
               'Has Table booking': [1],
               'Has Online delivery':[0],
               'Is delivering now':[1],
               'Switch to order menu':[1],
               'Price range' :[4],
               'Aggregate rating':[4.9],
               'Rating color':[1],
               'Rating text':[1],
               'Votes': [365],
               'Country' : [6]}

df_pred = pd.DataFrame(data_Predict,index=[0])
df_pred

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

rfr = RandomForestClassifier(random_state=0)

# Define a grid of hyperparameter 'params_
param_grid = {
    'n_estimators': [50, 100, 200], 
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_rf = GridSearchCV(estimator=rfr, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

grid_rf.fit(X_train, y_train)
print("Best parameters found: ", grid_rf.best_params_)
print("Best score found: ", grid_rf.best_score_)

In [ ]:
# Predicting the result using the best model. 
new_pred = best_rfr.predict(df_pred)
print(new_pred)

In [ ]:
from joblib import dump, load

# saving the model. 
dump(best_rf, 'best_rf.joblib') 

# Loading the model
best_rf_from_joblib = load('best_rf.joblib') 

# using the loaded model to make predictions
best_rf_from_joblib.predict(X1_test)